In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
archivos=[
    'poemas_edgar_allan_poe.txt',
    'Pablo Neruda.txt'
]

In [4]:
textos=[]
etiquetas=[]

In [5]:
for etiqueta, f in enumerate(archivos):
    print(f"{f} corresponde a: {etiqueta}")

    with open(f, 'r',encoding='utf-8') as archivo:
        for line in archivo:
            line = line.rstrip().lower()
            if line:
                line = line.translate(str.maketrans('','',string.punctuation))
                textos.append(line)
                etiquetas.append(etiqueta)
                

poemas_edgar_allan_poe.txt corresponde a: 0
Pablo Neruda.txt corresponde a: 1


In [22]:
#textos

In [23]:
#etiquetas


In [24]:
train_text,test_text,ytrain,ytest= train_test_split(textos,etiquetas,test_size=0.1,random_state=1)

In [25]:
len(ytrain), len(ytest)

(14481, 1610)

In [26]:
indice=1
indicepalabras={'<unk>':0}#se usar para asignar 0 a las palabras desconocidas en el entrenamiento

In [27]:
for txt in train_text:
    tokens=txt.split()
    for token in tokens:
        if token not in indicepalabras:
            indicepalabras[token]=indice
            indice+=1

In [28]:
#indicepalabras

In [29]:
#convertir a enteros
train_text_int=[]
test_text_int=[]

for txt in train_text:
    tokens=txt.split()
    linea_entero=[indicepalabras[token]for token in tokens]
    train_text_int.append(linea_entero)

for txt in test_text:
    tokens=txt.split()
    linea_entero=[indicepalabras.get(token,0)for token in tokens] #si esta la palabra en el diccionario toma su numero y si no esta toma el 0
    test_text_int.append(linea_entero)

In [30]:
v= len(indicepalabras)

a0= np.ones((v,v))
pi0= np.ones(v)

a1= np.ones((v,v))
pi1= np.ones(v)

In [31]:
#cuenta el numero de palabras con las que una frase empezo y cuantas veces siguio la otra palabra
def c_counts (text_int, a, pi):

    for tokens in text_int:
        last_idx= None
        for idx in tokens:
            if last_idx is None:
                pi[idx]+=1
            else:
                a[last_idx, idx]+=1
            last_idx = idx

c_counts([t for t,y in zip(train_text_int, ytrain)if y==0],a0,pi0)
c_counts([t for t,y in zip(train_text_int, ytrain)if y==1],a1,pi1)


In [32]:
#Normalizar a y pi para que sean matrices de probabilidad

a0/= a0.sum(axis=1, keepdims=True)
pi0/=pi0.sum()

a1/=a1.sum(axis=1,keepdims=True)
pi1/=pi1.sum()



In [42]:
#pasar a logaritmos para que no sean numeros muy pequeños
log_a0=np.log(a0)
log_pi0=np.log(pi0)

log_a1=np.log(a1)
log_pi1=np.log(pi1)

In [43]:
count0= sum(y==0 for y in ytrain) #cuenta etiquetas de clase 0 en ytrain
count1= sum(y==1 for y in ytrain) #cuenta etiquetas de clase 1 en ytrain
total = len(ytrain)#cantidad total de ejemplos de entrenamiento
p0= count0/total #probabilidad a priori de clase 0
p1= count1/total  #probabilidad a priori de clase 1
logp0=np.log(p0) #log de la probabilidad a priori clase 0
logp1=np.log(p1) #log de la probabilidad a priori clase 1

p0,p1#imprime las probabilidad a priori de ambas clases


(0.002969408190042124, 0.9970305918099579)

In [46]:
#construir clasificador
class clasificador:
    def __init__(self, logas, logpis, logpriors):
        self.logas =  logas
        self.logpis = logpis
        self.logpriors = logpriors
        self.K= len(logpriors)#numero de clases
    def _compute_log_likelihood(self, input_, class_):
        loga= self.logas[class_]
        logpi= self.logpis[class_]
        last_idx= None
        logprob= 0
        for idx in input_:
            if last_idx is None:
                logprob += logpi[idx]
            else:
                logprob += loga[last_idx, idx]
            last_idx= idx
        return logprob
    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            #calcula los logaritmos de las probabilidades posteriores para cada clase
            posteriors = [self._compute_log_likelihood(input_,c)+self.logpriors[c]\
                          for c in range(self.K)]
            #elige la clase con la mayor probabilidad posterior como la prediccion 
            pred=np.argmax(posteriors)
            predictions[i]=pred
        return predictions

In [47]:
#cada arreglo debe estar en orden ya que se asumen que las clases indexan estas listas

clf = clasificador([log_a0,log_a1],[log_pi0,log_pi1],[logp0,logp1])

In [50]:
ptrain= clf.predict(train_text_int)
print(f"Train acc: {np.mean(ptrain == ytrain)}")

Train acc: 0.9985498239071887


In [51]:
ptest= clf.predict(test_text_int)
print(f"Train acc: {np.mean(ptest == ytest)}")

Train acc: 0.9956521739130435
